In [1]:
#Use 3.12.4
import os
import markdown
import torch
from huggingface_hub import login
from sentence_transformers import SentenceTransformer

from langchain import FAISS
from langchain.text_splitter import SpacyTextSplitter 
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader, TextLoader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models.base import BaseChatModel
from langchain.schema import HumanMessage, AIMessage, SystemMessage

from langchain.docstore.in_memory import InMemoryDocstore
from langchain.docstore.document import Document
import faiss
import numpy as np

key_path = '/Users/jaesolshin/key/HF_TOKEN.txt'
os.environ["HF_TOKEN"] = open(key_path, 'r', encoding='utf-8').read()
login(os.environ["HF_TOKEN"], add_to_git_credential=True)

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (osxkeychain).
Your token has been saved to /Users/jaesolshin/.cache/huggingface/token
Login successful


In [22]:
classification_system_prompt = """

소셜 미디어 중재 시스템으로서 사용자 댓글을 분류하여야 합니다.
주제와 관련된 의견을 분석하여 다음 범주 중 하나로 분류합니다:

Positive(기쁨, 황홀, 신뢰, 지지, 공감, 기대 등 긍정적 감정이나 태도 )
Negetive(분노, 공포, 슬픔, 혐오, 짜증, 지루함 등 부정적 감정이나 태도)

이 의견이 위의 범주 중 어느 것에도 부합하지 않는 경우, 다음과 같이 분류합니다:
Neutral(중립)

설명 없이 카테고리만 숫자형식의 응답으로 제공합니다.

'2' (Positive), '1' (Neutral), '0' (Negetive)
"""

classification_template = """
아티스트: 뉴진스
댓글: 지금까지 위버스에 돈 다 쳐바른 내가 한심하다 팬들 돈 뽑아먹으니까 좋니~?	
클래스: 0

아티스트: 뉴진스
댓글: 혜인아 너 왜케 예뻐 .. ? 수퍼내추럴에서 또한번 어텐션만큼 찢었어 ㅠ ㅠ 사랑해...	
클래스: 2

아티스트: aespa
댓글: 나비스 나올 때 엑소 왓이즈러브 생각나네
클래스: 1

Topic: {group}
Comment: {comment}
Class:
"""

# 감정분류를 위한 주제와 댓글을 정의
sample_group= "aespa"
sample_comment = "aespa Forever커버곡좋다꽃길만걷고올해음악방송1위가자"

# 주제와 댓글을 바탕으로 분류 프롬프트 생성
prompt = classification_template.format(group=sample_group, comment=sample_comment)

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers

model_id = "beomi/gemma-ko-2b"
dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained("beomi/gemma-ko-2b")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="mps",
    torch_dtype=dtype,
)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=150)

In [21]:
tokenizer.decode(outputs[0]).split('Class:')[1]

'\nComment: 엑소 왓이즈러브 뮤비보고 울었다 ㅠㅠ 엑소 왓이즈러브 뮤비보고 울었다 ㅠㅠ 엑소 왓이즈러브 뮤비보고 울었다 ㅠㅠ 엑소 왓이즈러브 뮤비보고 울었다 ㅠㅠ 엑소 왓이즈러브 뮤비보고 울었다 ㅠㅠ 엑소 왓이즈러브 뮤비보고 울었다 ㅠㅠ 엑소 왓이즈러브 뮤비보고 울었다 ㅠㅠ 엑소 왓이즈러'

In [24]:
answer = tokenizer.decode(outputs[0]).split('Class')[1]
print(answer)

:
Comment: 엑소 왓이즈러브 뮤비보고 울었다 ㅠㅠ 엑소 왓이즈러브 뮤비보고 울었다 ㅠㅠ 엑소 왓이즈러브 뮤비보고 울었다 ㅠㅠ 엑소 왓이즈러브 뮤비보고 울었다 ㅠㅠ 엑소 왓이즈러브 뮤비보고 울었다 ㅠㅠ 엑소 왓이즈러브 뮤비보고 울었다 ㅠㅠ 엑소 왓이즈러브 뮤비보고 울었다 ㅠㅠ 엑소 왓이즈러


In [17]:
inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

In [16]:
model.generate(prompt)

AttributeError: 'str' object has no attribute 'shape'

In [26]:
# 시스템 프롬프트와 classification_template를 하나의 프롬프트로 결합
classification_system_prompt = """
소셜 미디어 중재 시스템으로서 사용자 댓글을 분류하여야 합니다.
주제와 관련된 의견을 분석하여 다음 범주 중 하나로 분류합니다:

Positive(기쁨, 황홀, 신뢰, 지지, 공감, 기대 등 긍정적 감정이나 태도 )
Negative(분노, 공포, 슬픔, 혐오, 짜증, 지루함 등 부정적 감정이나 태도)

이 의견이 위의 범주 중 어느 것에도 부합하지 않는 경우, 다음과 같이 분류합니다:
Neutral(중립)

설명 없이 카테고리만 숫자 형식으로 응답합니다:

'2' (Positive), '1' (Neutral), '0' (Negative)
"""

classification_template = """
아티스트: 뉴진스
댓글: 지금까지 위버스에 돈 다 쳐바른 내가 한심하다 팬들 돈 뽑아먹으니까 좋니~?	
클래스: 0

아티스트: 뉴진스
댓글: 혜인아 너 왜케 예뻐 .. ? 수퍼내추럴에서 또한번 어텐션만큼 찢었어 ㅠ ㅠ 사랑해...	
클래스: 2

아티스트: aespa
댓글: 나비스 나올 때 엑소 왓이즈러브 생각나네
클래스: 1

아티스트: {group}
댓글: {comment}
클래스:
"""

# 감정 분류를 위한 주제와 댓글을 정의
sample_group = "aespa"
sample_comment = "사랑해"

# 프롬프트 생성
prompt = classification_system_prompt + classification_template.format(group=sample_group, comment=sample_comment)

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "beomi/gemma-ko-2b"
dtype = torch.bfloat16


# 입력 텍스트를 토큰화하고 모델에 입력
inputs = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(input_ids=inputs, max_new_tokens=150)

# 결과 텍스트 디코딩
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# 클래스 값 추출
if '클래스:' in generated_text:
    answer = generated_text.split('클래스:')[1].strip().split()[0]  # '클래스:' 이후 첫 번째 값 추출
else:
    answer = "No class found"

print(f"Predicted class: {answer}")

Predicted class: 0


In [27]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# 예시 목록 정의 (Few-shot 샘플)
examples = [
    {"group": "뉴진스", "comment": "지금까지 위버스에 돈 다 쳐바른 내가 한심하다 팬들 돈 뽑아먹으니까 좋니~?", "class": "0"},
    {"group": "뉴진스", "comment": "혜인아 너 왜케 예뻐 .. ? 수퍼내추럴에서 또한번 어텐션만큼 찢었어 ㅠ ㅠ 사랑해...", "class": "2"},
    {"group": "aespa", "comment": "나비스 나올 때 엑소 왓이즈러브 생각나네", "class": "1"},
]

# 기본 예시를 활용한 few-shot 프롬프트 템플릿 정의
example_template = """
아티스트: {group}
댓글: {comment}
클래스: {class}
"""

# PromptTemplate을 사용해 각 예시를 구성하는 템플릿을 정의
example_prompt = PromptTemplate(
    input_variables=["group", "comment", "class"],
    template=example_template,
)

# 주어진 예시들과 함께 사용될 FewShotPromptTemplate 정의
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,  # 위에서 정의한 예시 리스트
    example_prompt=example_prompt,  # 각 예시에 대한 프롬프트 템플릿
    prefix="""
소셜 미디어 중재 시스템으로서 사용자 댓글을 분류하여야 합니다.
주제와 관련된 의견을 분석하여 다음 범주 중 하나로 분류합니다:

Positive(기쁨, 황홀, 신뢰, 지지, 공감, 기대 등 긍정적 감정이나 태도 )
Negative(분노, 공포, 슬픔, 혐오, 짜증, 지루함 등 부정적 감정이나 태도)

이 의견이 위의 범주 중 어느 것에도 부합하지 않는 경우, 다음과 같이 분류합니다:
Neutral(중립)

설명 없이 카테고리만 숫자 형식으로 응답합니다:

'2' (Positive), '1' (Neutral), '0' (Negative)

다음은 예시입니다:
""",  # 프롬프트 앞부분에 들어갈 시스템 지시사항
    suffix="""
아티스트: {group}
댓글: {comment}
클래스:
""",  # 실제 주어진 데이터에 대해 모델이 분류해야 하는 질문
    input_variables=["group", "comment"],  # 프롬프트에서 사용할 변수
)

# 실제 사용할 데이터
sample_group = "aespa"
sample_comment = "aespa Forever커버곡좋다꽃길만걷고올해음악방송1위가자"

# 최종 프롬프트 생성
final_prompt = few_shot_prompt.format(group=sample_group, comment=sample_comment)

# 생성된 프롬프트 출력
print(final_prompt)



소셜 미디어 중재 시스템으로서 사용자 댓글을 분류하여야 합니다.
주제와 관련된 의견을 분석하여 다음 범주 중 하나로 분류합니다:

Positive(기쁨, 황홀, 신뢰, 지지, 공감, 기대 등 긍정적 감정이나 태도 )
Negative(분노, 공포, 슬픔, 혐오, 짜증, 지루함 등 부정적 감정이나 태도)

이 의견이 위의 범주 중 어느 것에도 부합하지 않는 경우, 다음과 같이 분류합니다:
Neutral(중립)

설명 없이 카테고리만 숫자 형식으로 응답합니다:

'2' (Positive), '1' (Neutral), '0' (Negative)

다음은 예시입니다:



아티스트: 뉴진스
댓글: 지금까지 위버스에 돈 다 쳐바른 내가 한심하다 팬들 돈 뽑아먹으니까 좋니~?
클래스: 0



아티스트: 뉴진스
댓글: 혜인아 너 왜케 예뻐 .. ? 수퍼내추럴에서 또한번 어텐션만큼 찢었어 ㅠ ㅠ 사랑해...
클래스: 2



아티스트: aespa
댓글: 나비스 나올 때 엑소 왓이즈러브 생각나네
클래스: 1



아티스트: aespa
댓글: aespa Forever커버곡좋다꽃길만걷고올해음악방송1위가자
클래스:



In [28]:
# 입력 텍스트를 토큰화하고 모델에 입력
inputs = tokenizer.encode(final_prompt, return_tensors="pt").to(model.device)
outputs = model.generate(input_ids=inputs, max_new_tokens=150)

# 결과 텍스트 디코딩
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# 클래스 값 추출
if '클래스:' in generated_text:
    answer = generated_text.split('클래스:')[1].strip().split()[0]  # '클래스:' 이후 첫 번째 값 추출
else:
    answer = "No class found"

print(f"Predicted class: {answer}")

Predicted class: 0
